In [ ]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA

include(srcdir("nodeconstructor.jl"))
include(srcdir("env.jl"));

In [ ]:
CM = [0.0  1.0
 -1.0  0.0]

parameters = Dict()
source_list = []

source = Dict()

source["pwr"] = 45000.0
source["v_rip"] = 0.01556109320329396
source["vdc"] = 750
source["i_rip"] = 0.10108821490394984
source["fltr"] = "LCL"
source["R1"] = 0.4022094955070556
source["R2"] = 0.4022094955070556
source["R_C"] = 0.0006447094780419011
source["L1"] = 0.001005523738767639
source["L2"] = 0.001005523738767639
source["C"] = 2.302533850149647e-5;

push!(source_list, source);

load_list = []
load = Dict()

load["impedance"] = "RLC"
load["R"] = 30236.0;
load["L"] = 57.042;
load["C"] = 39.18;
push!(load_list, load);

cable_list = []

cable = Dict()
cable["R"] = 6.84059
cable["L"] = 0.00250127
cable["C"] = 3.7898e-6;
push!(cable_list, cable);

parameters["source"] = source_list
parameters["cable"] = cable_list
parameters["load"] = load_list;
parameters["grid"] = Dict("fs" => 10000.0, "phase" => 1, "v_rms" => 230);

In [ ]:
parameters

In [ ]:
nc = NodeConstructor(num_sources=1, num_loads=1, parameters=parameters, CM=CM);

In [ ]:
A, B, C, D = get_sys(nc);

In [ ]:
limits = Dict("i_lim" => 20, "v_lim" => 600)

In [ ]:
states = get_states(nc)
norm_array = []
for state_name in states
    if startswith(state_name, "i")
        push!(norm_array, limits["i_lim"])
    elseif startswith(state_name, "u")
        push!(norm_array, limits["v_lim"])
    end
end

In [ ]:
norm_array

In [ ]:
ns = length(A[1,:])
na = length(B[1,:])

In [ ]:
# time step
ts = 1e-5
V_source = 300

In [ ]:
x0 = [ 0.0 for i = 1:length(A[1,:]) ]
Ad = exp(A*ts)
Bd = A \ (Ad - C) * B

In [ ]:
env = SimEnv(A=A, B=B, C=C, Ad=Ad, Bd=Bd, norm_array=norm_array, x0=x0, v_dc=V_source, ts=rationalize(ts), convert_state_to_cpu=true);

In [ ]:
env.reward = reward_func("Power_exp", env)

In [ ]:
state(env)

In [ ]:
policy = RandomPolicy(action_space(env))

In [ ]:
[policy(env) for _ in 1:10]

In [ ]:
run(policy, env)

In [ ]:
state(env)

In [ ]:
reset!(env)

In [ ]:
env(-1)

In [ ]:
u[0] = 230